# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [69]:
import os
import datetime
import re
#%pip install requests beautifulsoup4
#%pip install requests
# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [70]:
# Use this cell for any import statements you add



---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [71]:
artists = {'robyn':"https://www.azlyrics.com/r/robyn.html",
           'cher':"https://www.azlyrics.com/c/cher.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: <!-- Delete this comment and put your answer here. --> 


In [72]:
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from collections import defaultdict
import requests, time, random

url_stub = "https://www.azlyrics.com"
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    try:
        r = requests.get(
            artist_page,
            headers={"User-Agent": "Mozilla/5.0 (compatible; ADS509-lyrics/1.0)"},
            timeout=30,
        )
        r.raise_for_status()
    except Exception as e:
        print(f"[WARN] Failed artist page for {artist}: {e}")
        continue

    time.sleep(5 + 10*random.random())

    soup = BeautifulSoup(r.text, "html.parser")

    # On AZLyrics, song links live under the album list. Grab all <a> that point to /lyrics/
    links = []
    for a in soup.select("a[href]"):
        href = a.get("href", "")
        if "/lyrics/" in href or href.startswith("../lyrics/") or href.startswith("/lyrics/"):
            full = urljoin(artist_page, href)
            # normalize to AZLyrics domain
            if "azlyrics.com" in full:
                links.append(full)

    # Dedup & store
    unique_links = []
    seen = set()
    for u in links:
        if u not in seen:
            seen.add(u)
            unique_links.append(u)

    lyrics_pages[artist].extend(unique_links)
    print(f"[COLLECT] {artist}: found {len(unique_links)} song pages")


[COLLECT] robyn: found 104 song pages
[COLLECT] cher: found 334 song pages


Let's make sure we have enough lyrics pages to scrape. 

In [73]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [74]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For robyn we have 104.
The full pull will take for this artist will take 0.29 hours.
For cher we have 334.
The full pull will take for this artist will take 0.93 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [75]:
def generate_filename_from_link(link):
    if not link:
        return None
    
    # chain replacements on the SAME variable
    name = link.replace("https", "").replace("http", "")
    name = name.replace(".html", "")
    name = name.replace("/lyrics/", "")
    name = name.replace("://", "").replace(".", "_").replace("/", "_")
    name = name + ".txt"
    return name



In [76]:
import shutil

# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [77]:
url_stub = "https://www.azlyrics.com" 
start = time.time()
from pathlib import Path
total_pages = 0 
Path("lyrics").mkdir(exist_ok=True)
for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    artist_dir = Path("lyrics") / artist
    artist_dir.mkdir(parents=True, exist_ok=True)

    pulled = 0

    # 2. Iterate over the lyrics pages
    for url in lyrics_pages[artist]:
        # optional cap to “at least 20”
        # if pulled >= 20: 
        #     break

        # 3) Request the lyrics page (polite crawling)
        try:
            resp = requests.get(
                url,
                headers={"User-Agent": "Mozilla/5.0 (compatible; ADS509-lyrics/1.0)"},
                timeout=30,
            )
            resp.raise_for_status()
        except Exception as e:
            print(f"[WARN] Failed to fetch {url} -> {e}")
            time.sleep(1 + 2*random.random())
            continue

        # Always sleep after a request
        time.sleep(5 + 10*random.random())

        # 4) Extract the title and lyrics from the page
        soup = BeautifulSoup(resp.text, "html.parser")

        # title candidates
        title = None
        if soup.find("h1"):
            title = soup.find("h1").get_text(strip=True)
        if not title:
            og = soup.find("meta", attrs={"property": "og:title"})
            if og and og.get("content"):
                title = og["content"].strip()
        if not title and soup.title:
            title = soup.title.get_text(strip=True)
        if not title:
            title = "Untitled"

        # lyrics extraction:
        # Many AZLyrics-like pages keep lyrics in a <div> without id/class.
        # We’ll choose the longest such block as “lyrics”.
        lyrics = ""
        candidates = []
        for div in soup.find_all("div"):
            if div.get("class") is None and div.get("id") is None:
                text = div.get_text("\n", strip=True)
                if text and len(text.split()) > 30:
                    candidates.append(text)

        if candidates:
            lyrics = max(candidates, key=len)
        else:
            # Fallback: whole page text (worst case)
            lyrics = soup.get_text("\n", strip=True)

        # 5) Write out title, two returns, and the lyrics
        fname = generate_filename_from_link(url)
        out_path = artist_dir / fname

        # skip if you already have it
        if out_path.exists():
            print(f"[SKIP] {out_path.name} already exists.")
            continue

        try:
            with open(out_path, "w", encoding="utf-8") as f:
                f.write(f"{title}\n\n{lyrics}")
            pulled += 1
            print(f"[OK] Saved {artist}/{out_path.name}")
        except Exception as e:
            print(f"[ERR] Could not write {out_path} -> {e}")
    # 3. Request the lyrics page. 
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
    # 4. Extract the title and lyrics from the page.
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
    
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
    

[OK] Saved robyn/www_azlyrics_comrobyn_bumpyride.txt
[OK] Saved robyn/www_azlyrics_comrobyn_inmyheart.txt
[OK] Saved robyn/www_azlyrics_comrobyn_youvegotthatsomething.txt
[OK] Saved robyn/www_azlyrics_comrobyn_doyouknowwhatittakes.txt
[OK] Saved robyn/www_azlyrics_comrobyn_thelasttime.txt
[OK] Saved robyn/www_azlyrics_comrobyn_justanothergirlfriend.txt
[OK] Saved robyn/www_azlyrics_comrobyn_dontwantyouback.txt
[OK] Saved robyn/www_azlyrics_comrobyn_doyoureallywantmeshowrespect.txt
[OK] Saved robyn/www_azlyrics_comrobyn_how.txt
[OK] Saved robyn/www_azlyrics_comrobyn_herewego.txt
[OK] Saved robyn/www_azlyrics_comrobyn_wheredidourlovego.txt
[OK] Saved robyn/www_azlyrics_comrobyn_robynishere.txt
[OK] Saved robyn/www_azlyrics_comrobyn_iwish.txt
[OK] Saved robyn/www_azlyrics_comrobyn_showmelove.txt
[OK] Saved robyn/www_azlyrics_comrobyn_play.txt
[OK] Saved robyn/www_azlyrics_comrobyn_myonlyreason.txt
[OK] Saved robyn/www_azlyrics_comrobyn_underneaththeheart.txt
[OK] Saved robyn/www_azlyrics_

In [78]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 1.27 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [79]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [80]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For cher we have 334 files.
For cher we have roughly 88579 words, 3761 are unique.
For robyn we have 104 files.
For robyn we have roughly 31585 words, 2276 are unique.
